<span class="mathmacros" style="display:none;">
$\def\bra#1{\mathinner{\langle{#1}|}}
\def\ket#1{\mathinner{|{#1}\rangle}}
\def\braket#1{\mathinner{\langle{#1}\rangle}}
\newcommand{\mdef}{\overset{\mathrm{def}}{=}}
\newcommand{\bm}{\mathbf}
\newcommand{\inv}[1]{#1^{-1}}   % Inverse Matrix
\newcommand{\invt}[1]{#1^{-T}}  % Inverse Transposed Matrix
\renewcommand{\nl}{\\&\phantom{{}={}}}% Newline In aligned equations
\newcommand{\pfr}[2]{\frac{\pp #1}{\pp #2}}      % Partial derivative
\newcommand{\dfr}[2]{\frac{\dd #1}{\dd #2}}      % Total derivative
\newcommand{\pp}{\partial}
\DeclareMathOperator{\Var}{Var}
\DeclareMathOperator{\det}{det}
\DeclareMathOperator{\tr}{tr}
\DeclareMathOperator{\sgn}{sgn}
\DeclareMathOperator{\adj}{adj}
\DeclareMathOperator{\ii}{i}
\DeclareMathOperator{\dd}{d}
\DeclareMathOperator{\rhs}{RHS}
\DeclareMathOperator{\lhs}{LHS}
\newcommand{\nl}{\\&\phantom{={}}}
\DeclareMathOperator{\E}{E}
\DeclareMathOperator{\Cov}{Cov}
\DeclareMathOperator{\Beta}{B}
\DeclareMathOperator{\Bdist}{Beta}$
</span>

## Optimization process

### Initial state
Initially we have a product density matrix $\rho=\bigotimes \rho_i$

### Optimization steps

Suppose $h$ is a Hermitian matrix, then $U=e^{ih}$ is unitary. The optimization is iteratively applying $U_i=e^{ih_i}$ at step $i$ to give a better $$\rho_{i+1}=e^{ih_i}\rho_i e^{-ih_i}$$
to minimize
$$f(h_i)=\tr[H^2U_i\rho_i U_i^+]-\tr^2[HU_i\rho_i U_i^+]$$

The optimization at each step is using gradient $M=\nabla_h f$ to give a descent direction. Then consider $h_i=xM$, we can find first and second order derivative over $x$, which gives an estimation on best step size $x=-f'/f''$.

This process is similar for optimization of local Unitaries

#### Approximation to $\exp$
In every optimization step, once we have an $h$ we should apply $\exp(i h)$ to operators. If we replace the exact `scipy.linalg.expm` with pade approximation $$e^x\approx \frac{1+x/2}{1-x/2},$$ the optimization steps are much faster.

## Time complexity
Parameters: Depth $D$ of circuit, length $L$ of chain. General conclusions:
+ Number of local $U$ is $N_u=DL/l$
+ Local unitaries are $m\times m$, where $m=2^l=4$ 
+ Dense matrix is $n\times n$, where $n=2^L$

### Dense optimization
We have dense $\rho$ and $H$ with dimension $n\times n$. The initial contraction can be amortized to every block, and during one contraction cycle there are $N_u$ blocks to contract. In every step, the time of transform a pair of slots is proportional to applying single $U$ to huge matrix, i.e. $mn^2$. Actually, we will apply four times($\rho$ twice and $H$ twice). Contract to hole is also $mn^2$. Time complexity for one cycle is
$$\Theta(mn^2N_u)=\Theta(2^{2L+l}DL/l)$$

### Sparse optimization
We make the assumption that $L>D$.
#### MPO
Except for the MPO part, it is similar to dense optimization, but rotated by $\pi/2$. There are $D-1$ free 2-legs maintained for every half of the system, and one M-leg for the MPO. The dense matrix maintained has dimension $2^{2(D-1)}M$. Similar to the analysis in dense optimization, the time complexity for one cycle is
$$\Theta(2^{2(D-1)}MmN_u)$$

The speed boost of sparse mpo is $k\approx 4^{L-D}$. If $L$ is larger than $D$, this is a win!

#### ~~Cones method~~
It seems that cone method has weak overlapping between different terms. We decompose $H^2$ into $L^2$ terms and later collect effect of each one. Considering dependency of each term, it only depends on $LD$ terms. Naively, for each term the contraction is proportional to $D^2$. Consider full contraction for a single hole, the time is $LDD^22^Dm$

### What is effect of optimization order?
Can not easily predict...